# Download Data

In [ ]:
!wget -q https://hkustconnect-my.sharepoint.com/:u:/g/personal/nnanda_connect_ust_hk/EfREjZqiZTlPqhqUPICBbPABdlgPumlaUVxPncm-_9aWIw?download=1 -O "Project 1 - data.zip"
!unzip -q "Project 1 - data.zip"

# Import Libraries

In [ ]:
!pip -q install keras-layer-normalization

In [ ]:
import os
import nltk
import math
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, BatchNormalization,\
    Activation, Input, Add, Concatenate, Embedding, Conv1D, MaxPool1D,\
    Flatten, LSTM, Bidirectional, MaxPooling1D, SimpleRNN, GRU, \
    SpatialDropout1D
from keras_layer_normalization import LayerNormalization
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stopwords = set(stopwords.words("english"))
ps = PorterStemmer()

# Data Loader

In [ ]:
def load_data(split_name='train', columns=['text', 'stars']):
    try:
        print(f"select [{', '.join(columns)}] columns from the {split_name} split")
        df = pd.read_csv(f'data_2021_spring/{split_name}.csv')
        df = df.loc[:,columns]
        print("succeed!")
        return df
    except:
        print("Failed, then try to ")
        print(f"select all columns from the {split_name} split")
        df = pd.read_csv(f'data_2021_spring/{split_name}.csv')
        return df

In [ ]:
train_df = load_data('train', columns=['full'])
valid_df = load_data('valid', columns=['full'])
test_df = load_data('test', columns=['full'])

select [full] columns from the train split
Failed, then try to 
select all columns from the train split
select [full] columns from the valid split
Failed, then try to 
select all columns from the valid split
select [full] columns from the test split
Failed, then try to 
select all columns from the test split


# Feature Extractor

In [ ]:
def tokenize(text):
    """
    :param text: a doc with multiple sentences, type: str
    return a word list, type: list
    e.g.
    Input: 'Text mining is to identify useful information.'
    Output: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    """
    return nltk.word_tokenize(text)

def stem(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of stemmed words, type: list
    e.g.
    Input: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    Output: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    """

    return [ps.stem(token).lower() for token in tokens]

# Just for testing, was not used in tutorial - removing stopwords doesn't help much
def filter_stopwords(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of filtered tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    Output: ['text', 'mine', 'identifi', 'use', 'inform', '.']
    """
    ### equivalent code
    # results = list()
    # for token in tokens:
    #     if token not in stopwords and not token.isnumeric():
    #         results.append(token)
    # return results

    return [token for token in tokens if token not in stopwords and not token.isnumeric()]

In [ ]:
def get_feats_dict(feats, min_freq=-1, max_freq=-1, max_size=-1):
    """
    :param data: a list of features, type: list(list)
    :param min_freq: the lowest fequency that the fequency of a feature smaller than it will be filtered out, type: int
    :param max_freq: the highest fequency that the fequency of a feature larger than it will be filtered out, type: int
    :param max_size: the max size of feature dict, type: int
    return a feature dict that maps features to indices, sorted by frequencies
    # Counter document: https://docs.python.org/3.6/library/collections.html#collections.Counter
    """
    # count all features
    feat_cnt = Counter(feats) # ["text", "text", "mine"] --> {"text": 2, "mine": 1}
    if max_size > 0 and min_freq == -1 and max_freq == -1:
        valid_feats = ["<pad>", "<unk>"] + [f for f, cnt in feat_cnt.most_common(max_size-2)]
    else:
        valid_feats = ["<pad>", "<unk>"]
        for f, cnt in feat_cnt.most_common():
            if (min_freq == -1 or cnt >= min_freq) and \
                (max_freq == -1 or cnt <= max_freq):
                valid_feats.append(f)
    if max_size > 0 and len(valid_feats) > max_size:
        valid_feats = valid_feats[:max_size]
    print("Size of features:", len(valid_feats))
    
    # build a mapping from features to indices
    feats_dict = dict(zip(valid_feats, range(len(valid_feats))))
    return feats_dict

def get_index_vector(feats, feats_dict, max_len):
    """
    :param feats: a list of features, type: list
    :param feats_dict: a dict from features to indices, type: dict
    :param feats: a list of features, type: list
    return a feature vector,
    """
    # initialize the vector as all zeros
    vector = np.zeros(max_len, dtype=np.int64)
    for i, f in enumerate(feats):
        if i == max_len:
            break
        # get the feature index, return 1 (<unk>) if the feature is not existed
        f_idx = feats_dict.get(f, 1)
        vector[i] = f_idx
    return vector

# Create Input Features

In [ ]:
min_freq = 3

# load data
train_texts, train_labels = train_df["text"], train_df["stars"]
valid_texts, valid_labels = valid_df["text"], valid_df["stars"]

# extract features
train_tokens = [tokenize(text) for text in train_texts]
valid_tokens = [tokenize(text) for text in valid_texts]


train_stemmed = [stem(tokens) for tokens in train_tokens]
valid_stemmed = [stem(tokens) for tokens in valid_tokens]

# If stopwords not used
train_feats = train_stemmed
valid_feats = valid_stemmed

# filtering stopwords didn't help much
# train_feats = [filter_stopwords(tokens) for tokens in train_stemmed]
# valid_feats = [filter_stopwords(tokens) for tokens in valid_stemmed]

# build a mapping from features to indices
feats_dict = get_feats_dict(chain.from_iterable(train_feats), min_freq=min_freq)

Size of features: 9357


In [ ]:
max_len = 100

# build the feats_matrix
# convert each example to a index vector, and then stack vectors as a matrix
train_feats_matrix = np.vstack(
    [get_index_vector(f, feats_dict, max_len) for f in train_feats])
valid_feats_matrix = np.vstack(
    [get_index_vector(f, feats_dict, max_len) for f in valid_feats])

# convert labels to label_matrix
num_classes = max(train_labels)
# convert each label to a ont-hot vector, and then stack vectors as a matrix
train_label_matrix = keras.utils.to_categorical(train_labels-1, num_classes=num_classes)
valid_label_matrix = keras.utils.to_categorical(valid_labels-1, num_classes=num_classes)

# Word2Vec

Add word2vec embeddings and tune hyperparameters

In [ ]:
!wget -q https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
%cd fastText-0.9.2
!make

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
   creating: fastText-0.9.2/
   creating: fastText-0.9.2/.circleci/
  inflating: fastText-0.9.2/.circleci/cmake_test.sh  
  inflating: fastText-0.9.2/.circleci/config.yml  
  inflating: fastText-0.9.2/.circleci/gcc_test.sh  
  inflating: fastText-0.9.2/.circleci/pip_test.sh  
  inflating: fastText-0.9.2/.circleci/pull_data.sh  
  inflating: fastText-0.9.2/.circleci/python_test.sh  
  inflating: fastText-0.9.2/.circleci/run_locally.sh  
  inflating: fastText-0.9.2/.circleci/setup_circleimg.sh  
  inflating: fastText-0.9.2/.circleci/setup_debian.sh  
  inflating: fastText-0.9.2/.gitignore  
  inflating: fastText-0.9.2/CMakeLists.txt  
  inflating: fastText-0.9.2/CODE_OF_CONDUCT.md  
  inflating: fastText-0.9.2/CONTRIBUTING.md  
  inflating: fastText-0.9.2/LICENSE  
  inflating: fastText-0.9.2/MANIFEST.in  
  inflating: fastText-0.9.2/Makefile  
  inflating: fastText-0.9.2/README.md  
   creating: fastText-0.9.2/alignment/
  in

In [ ]:
!cp fasttext ../
%cd ..

/content


In [ ]:
with open('reviews.txt', 'w') as filehandle:
    for sent in train_feats:
        combined_sent = (" ").join(sent)
        filehandle.write('%s\n' % combined_sent)

In [ ]:
os.makedirs("models", exist_ok=True)

In [ ]:
!./fasttext skipgram -input ./reviews.txt -output models/word2vec -dim 100 -minCount 3 -epoch 10

Read 1M words
Number of words:  9356
Number of labels: 0
Progress: 100.0% words/sec/thread:   21276 lr:  0.000000 avg.loss:  2.333479 ETA:   0h 0m 0s


In [ ]:
embedding_matrix = np.zeros((len(feats_dict), 100), dtype=np.float32)
with open("models/word2vec.vec", "r") as f:
    n_words, n_dim = f.readline().split()
    n_words, n_dim = int(n_words), int(n_dim) 
    print("number of words:", n_words, "word dimension:", n_dim)
    while True:
        line = f.readline()
        if not line:
            break
        word, vec = line.split(" ", 1)
        word_idx = feats_dict.get(word, -1)
        if word_idx != -1:
            embedding_matrix[word_idx] = np.array(vec.split(), dtype=np.float32)

number of words: 9356 word dimension: 100


In [ ]:
print(embedding_matrix[:5])

[[ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00 

# RNN Builder

In [ ]:
def build_MLP_Emb(input_length, vocab_size, embedding_size,
              hidden_size, output_size,
              num_rnn_layers, num_mlp_layers,
              rnn_type="lstm",
              bidirectional=False,
              embedding_matrix=None,
              activation="tanh",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy",
              num_filters=128,
              kernel_size=2,
              strides=1):
    """
    :param input_length: the maximum length of sentences, type: int
    :param vocab_size: the vacabulary size, type: int
    :param embedding_size: the dimension of word representations, type: int
    :param hidden_size: the dimension of the hidden states, type: int
    :param output_size: the dimension of the prediction, type: int
    :param num_rnn_layers: the number of layers of the RNN, type: int
    :param num_mlp_layers: the number of layers of the MLP, type: int
    :param rnn_type: the type of RNN, type: str
    :param bidirectional: whether to use bidirectional rnn, type: bool
    :param activation: the activation type, type: str
    :param dropout_rate: the probability of dropout, type: float
    :param batch_norm: whether to enable batch normalization, type: bool
    :param l2_reg: the weight for the L2 regularizer, type: str
    :param loss: the training loss, type: str
    :param optimizer: the optimizer, type: str
    :param learning_rate: the learning rate for the optimizer, type: float
    :param metric: the metric, type: str
    return a RNN for text classification,
    # activation document: https://keras.io/activations/
    # dropout document: https://keras.io/layers/core/#dropout
    # embedding document: https://keras.io/layers/embeddings/#embedding
    # recurrent layers document: https://keras.io/layers/recurrent
    # batch normalization document: https://keras.io/layers/normalization/
    # losses document: https://keras.io/losses/
    # optimizers document: https://keras.io/optimizers/
    # metrics document: https://keras.io/metrics/
    """
    x = Input(shape=(input_length,))
    
    ################################
    ###### Word Representation #####
    ################################
    # word representation layer
    if embedding_matrix is not None:
        emb = Embedding(input_dim=vocab_size,
                        output_dim=embedding_size,
                        input_length=input_length,
                        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                        trainable=True)(x)
    else:
        emb = Embedding(input_dim=vocab_size,
                        output_dim=embedding_size,
                        input_length=input_length,
                        embeddings_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=0))(x)
    
    ################################
    ####### Recurrent Layers #######
    ################################
    # recurrent layers
    if rnn_type == "rnn":
        fn = SimpleRNN
    elif rnn_type == "lstm":
        fn = LSTM
    elif rnn_type == "gru":
        fn = GRU
    else:
        raise NotImplementedError
    h = emb

    # for i in range(num_rnn_layers):
    #     is_last = (i == num_rnn_layers-1)
    #     if bidirectional:
    #         h = Bidirectional(fn(hidden_size,
    #                              kernel_initializer=keras.initializers.glorot_uniform(seed=0),
    #                              recurrent_initializer=keras.initializers.Orthogonal(gain=1.0, seed=0),
    #                              return_sequences=not is_last))(h)
    #     else:
    #         h = fn(hidden_size,
    #                kernel_initializer=keras.initializers.glorot_uniform(seed=0),
    #                recurrent_initializer=keras.initializers.Orthogonal(gain=1.0, seed=0),
    #                return_sequences=not is_last)(h)
    #     h = Dropout(dropout_rate, seed=0)(h)

    conv = Conv1D(filters=num_filters, kernel_size=kernel_size, padding="valid", strides=strides)(h)

    conv = Activation("relu")(conv)
        
    maxpool = MaxPool1D(pool_size=(input_length-kernel_size)//strides+1)(conv)
    # maxpool = MaxPool1D()(conv)

    maxpool = Flatten()(maxpool)

    h = Dropout(dropout_rate)(maxpool)
    
    ################################
    #### Fully Connected Layers ####
    ################################
    # multi-layer perceptron
    for i in range(num_mlp_layers-1):
        new_h = Dense(hidden_size,
                      kernel_initializer=keras.initializers.he_normal(seed=0),
                      bias_initializer="zeros",
                      kernel_regularizer=keras.regularizers.l2(l2_reg))(h)
        # add batch normalization layer
        if batch_norm:
            new_h = BatchNormalization()(new_h)
        # add residual connection
        if i == 0:
            h = new_h
        else:
            h = Add()([h, new_h])
        # add activation
        h = Activation(activation)(h)
    y = Dense(output_size,
              activation="softmax",
              kernel_initializer=keras.initializers.he_normal(seed=0),
              bias_initializer="zeros")(h)
    
    # set the loss, the optimizer, and the metric
    if optimizer == "SGD":
        optimizer = keras.optimizers.SGD(lr=learning_rate)
    elif optimizer == "RMSprop":
        optmizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == "Adam":
        optmizer = keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        raise NotImplementedError
    model = Model(x, y)
    model.compile(loss=loss, optimizer=optimizer, metrics=[metric])
    
    return model

In [ ]:
model = build_MLP_Emb(input_length=max_len, vocab_size=len(feats_dict),
                  embedding_size=100, hidden_size=100, output_size=num_classes,
                  rnn_type="lstm", num_rnn_layers=0, bidirectional=True, num_mlp_layers=2,
                  embedding_matrix=embedding_matrix,
                  activation="tanh",
                  batch_norm=True,
                  l2_reg=0.005, dropout_rate=0.5)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 100)          935700    
_________________________________________________________________
conv1d (Conv1D)              (None, 99, 128)           25728     
_________________________________________________________________
activation (Activation)      (None, 99, 128)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 128)            0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0     

In [ ]:
def Emb_Lstm_Cnn_Mlp_modified(input_length=max_len, vocab_size=len(feats_dict), 
                     embedding_size=100, hidden_size=100, 
                     num_filters=100, kernel_size=2, strides=1, 
                     output_size=num_classes, learning_rate=0.001,
                     dropout_rate=0.5, recurrent_dropout_rate=0.5, 
                     train=False, optimizer="RMSprop"):
    x = Input(shape=(input_length,))
    if embedding_matrix is not None:
        emb = Embedding(input_dim=vocab_size,
                            output_dim=embedding_size,
                            input_length=input_length,
                            embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                            trainable=train)(x)
    else:
        emb = Embedding(input_dim=vocab_size,
                        output_dim=embedding_size,
                        input_length=input_length,
                        embeddings_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=0))(x)

    emb = SpatialDropout1D(dropout_rate)(emb)

    rec = Bidirectional(LSTM(hidden_size,
                        kernel_initializer=keras.initializers.glorot_uniform(seed=0),
                        recurrent_initializer=keras.initializers.Orthogonal(gain=1.0, seed=0),
                        return_sequences=True, dropout=dropout_rate, recurrent_activation="sigmoid"))(emb)

    h = Concatenate()([emb, rec])

    # h = Dropout(dropout_rate)(h)

    conv = Conv1D(filters=num_filters, kernel_size=kernel_size, padding="valid", strides=strides)(h)

    conv = Activation("relu")(conv)
        
    maxpool = MaxPool1D(pool_size=(input_length-kernel_size)//strides+1)(conv)
    # maxpool = MaxPool1D()(conv)

    maxpool = Flatten()(maxpool)

    maxpool = Dropout(dropout_rate)(maxpool) # testing

    y = Dense(output_size,
              activation="softmax",
              kernel_initializer=keras.initializers.he_normal(seed=0),
              bias_initializer="zeros")(maxpool)

    model = Model(x, y)
    # optimizer = keras.optimizers.Adam(lr=learning_rate)
    # optimizer = keras.optimizers.RMSprop(lr=learning_rate)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    # print(model.summary())
    return model

In [ ]:
model = Emb_Lstm_Cnn_Mlp_modified()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 100)     935700      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 100)     0           embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 100, 200)     160800      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

In [ ]:
os.makedirs("models", exist_ok=True)
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_word2vec_cnn_mlp.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

model = Emb_Lstm_Cnn_Mlp_modified()
word2vec_cnn_mlp_modified_history = model.fit(train_feats_matrix, train_label_matrix,
                                    validation_split=0.1,
                                    epochs=100, batch_size=100, verbose=1,
                                    callbacks=[checkpointer, earlystopping, reduce_lr])
model = keras.models.load_model(os.path.join("models", "weights_word2vec_cnn_mlp.hdf5"))

train_score = model.evaluate(train_feats_matrix, train_label_matrix,
                             batch_size=100)
valid_score = model.evaluate(valid_feats_matrix, valid_label_matrix,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("valid loss:", valid_score[0], "valid accuracy", valid_score[1])

Epoch 1/100
90/90 [==============================] - 39s 390ms/step - loss: 1.6706 - accuracy: 0.2796 - val_loss: 1.4983 - val_accuracy: 0.3390

Epoch 00001: val_accuracy improved from -inf to 0.33900, saving model to models/weights_word2vec_cnn_mlp.hdf5
Epoch 2/100
90/90 [==============================] - 34s 382ms/step - loss: 1.4393 - accuracy: 0.3923 - val_loss: 1.3628 - val_accuracy: 0.4460

Epoch 00002: val_accuracy improved from 0.33900 to 0.44600, saving model to models/weights_word2vec_cnn_mlp.hdf5
Epoch 3/100
90/90 [==============================] - 34s 383ms/step - loss: 1.3729 - accuracy: 0.4166 - val_loss: 1.2929 - val_accuracy: 0.4570

Epoch 00003: val_accuracy improved from 0.44600 to 0.45700, saving model to models/weights_word2vec_cnn_mlp.hdf5
Epoch 4/100
90/90 [==============================] - 34s 382ms/step - loss: 1.3154 - accuracy: 0.4503 - val_loss: 1.2278 - val_accuracy: 0.5080

Epoch 00004: val_accuracy improved from 0.45700 to 0.50800, saving model to models/w

In [ ]:
# model = build_MLP_Emb(input_length=max_len, vocab_size=len(feats_dict),
#                   embedding_size=100, hidden_size=100, output_size=num_classes,
#                   rnn_type="lstm", num_rnn_layers=0, bidirectional=True, num_mlp_layers=2,
#                   embedding_matrix=embedding_matrix,
#                   activation="tanh",
#                   batch_norm=True,
#                   l2_reg=0.005, dropout_rate=0.5)

os.makedirs("models", exist_ok=True)
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_word2vec_cnn_mlp.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=15,
    verbose=1)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=7, verbose=1)

model = build_MLP_Emb(input_length=max_len, vocab_size=len(feats_dict),
                  embedding_size=100, hidden_size=100, output_size=num_classes,
                  rnn_type="lstm", num_rnn_layers=0, bidirectional=True, num_mlp_layers=2,
                  embedding_matrix=embedding_matrix,
                  activation="relu",
                  batch_norm=True,
                  l2_reg=0.005, dropout_rate=0.5)
word2vec_cnn_mlp_history = model.fit(train_feats_matrix, train_label_matrix,
                                    validation_split=0.1,
                                    epochs=100, batch_size=100, verbose=1,
                                    callbacks=[checkpointer, earlystopping, reduce_lr])
model = keras.models.load_model(os.path.join("models", "weights_word2vec_cnn_mlp.hdf5"))

train_score = model.evaluate(train_feats_matrix, train_label_matrix,
                             batch_size=100)
valid_score = model.evaluate(valid_feats_matrix, valid_label_matrix,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("valid loss:", valid_score[0], "valid accuracy", valid_score[1])

Epoch 1/100
90/90 [==============================] - 7s 67ms/step - loss: 2.7007 - accuracy: 0.2596 - val_loss: 2.2209 - val_accuracy: 0.3790

Epoch 00001: val_accuracy improved from -inf to 0.37900, saving model to models/weights_word2vec_cnn_mlp.hdf5
Epoch 2/100
90/90 [==============================] - 6s 70ms/step - loss: 2.1314 - accuracy: 0.3880 - val_loss: 1.8221 - val_accuracy: 0.4770

Epoch 00002: val_accuracy improved from 0.37900 to 0.47700, saving model to models/weights_word2vec_cnn_mlp.hdf5
Epoch 3/100
90/90 [==============================] - 6s 72ms/step - loss: 1.7172 - accuracy: 0.4862 - val_loss: 1.6154 - val_accuracy: 0.4970

Epoch 00003: val_accuracy improved from 0.47700 to 0.49700, saving model to models/weights_word2vec_cnn_mlp.hdf5
Epoch 4/100
90/90 [==============================] - 6s 63ms/step - loss: 1.4553 - accuracy: 0.5440 - val_loss: 1.4702 - val_accuracy: 0.5220

Epoch 00004: val_accuracy improved from 0.49700 to 0.52200, saving model to models/weights_w